In [1]:
# Autoreload Python Modules
# https://gist.github.com/deehzee/86ad60d3d2e375013ec710234f88968c
%load_ext autoreload
%autoreload 2

In [73]:
from datetime import datetime
import pandas as pd
import pandas_datareader.data as web
import tulipy as ti
import ta
from highcharts import Highstock
from highcharts.highstock.highstock_helper import jsonp_loader

df = web.DataReader("aapl", 'morningstar', datetime(2016, 1, 1)).reset_index()
ohlc = df[['Date', 'Open', 'High', 'Low', 'Close']].values.tolist()
volume = df[['Date', 'Volume']].values.tolist()

date_df = df[['Date']].set_index('Date')
close = df['Close'].values

rsi_length = 14
rsi_df = pd.DataFrame({ 'RSI': ti.rsi(close, rsi_length) })
rsi_df.index = date_df.index[-len(rsi_df):]

k_array, d_array = ta.srsi(close, rsi_length, rsi_length, 3, 3)
srsi_df = pd.DataFrame({ 'K': k_array, 'D': d_array })
srsi_df.index = date_df.index[-len(srsi_df):]

hma_length = 9
hma_df = pd.DataFrame({ 'HMA': ti.hma(close, hma_length) })
hma_df.index = date_df.index[-len(hma_df):]

tsi_array, tsi_signal_array = ta.tsi(close, 25, 13, 13)
tsi_df = pd.DataFrame({ 'TSI': tsi_array, 'TSI Signal': tsi_signal_array })
tsi_df.index = date_df.index[-len(tsi_df):]

indicators = pd.concat([date_df, rsi_df, srsi_df, hma_df, tsi_df], axis=1).reset_index()

rsi = indicators[['Date', 'RSI']].values.tolist()
k = indicators[['Date', 'K']].values.tolist()
d = indicators[['Date', 'D']].values.tolist()
hma = indicators[['Date', 'HMA']].values.tolist()
tsi = indicators[['Date', 'TSI']].values.tolist()
tsi_signal = indicators[['Date', 'TSI Signal']].values.tolist()

groupingUnits = [
    ['week', [1]], 
    ['month', [1, 2, 3, 4, 6]]
]

H = Highstock()

H.add_data_set(volume, 'column', 'Volume', yAxis = 1, dataGrouping = {
    'units': groupingUnits
})

H.add_data_set(rsi, 'line', 'RSI', yAxis = 2, dataGrouping = {
    'units': groupingUnits
})

H.add_data_set(k, 'line', 'K', yAxis = 2, dataGrouping = {
    'units': groupingUnits
})

H.add_data_set(d, 'line', 'D', yAxis = 2, dataGrouping = {
    'units': groupingUnits
})

H.add_data_set(tsi, 'line', 'TSI', yAxis = 2, dataGrouping = {
    'units': groupingUnits
})

H.add_data_set(tsi_signal, 'line', 'TSI Signal', yAxis = 2, dataGrouping = {
    'units': groupingUnits
})

H.add_data_set(hma, 'line', 'HMA', dataGrouping = {
    'units': groupingUnits
})

H.add_data_set(ohlc, 'candlestick', 'AAPL', dataGrouping = {
    'units': groupingUnits
})

options = {
    'rangeSelector': {
                'selected': 1
            },

    'title': {
        'text': 'AAPL Historical'
    },
    
    'chart': {
        'height': 600
    },

    'yAxis': [{
        'labels': {
            'align': 'right',
            'x': -3
        },
        'title': {
            'text': 'OHLC'
        },
        'height': '70%',
        'lineWidth': 2
    }, {
        'labels': {
            'align': 'right',
            'x': -3
        },
        'title': {
            'text': 'Volume'
        },
        'top': '60%',
        'height': '15%',
        'offset': 0,
        'lineWidth': 2
    }, {
        'labels': {
            'align': 'right',
            'x': -3
        },
        'title': {
            'text': 'RSI'
        },
        'top': '75%',
        'height': '25%',
        'offset': 0,
        'lineWidth': 2
    }],
}

H.set_dict_options(options)

H

